In [3]:
import json
import pandas as pd
import numpy as np
from transformers import LlamaTokenizer, LlamaForCausalLM
import torch

from transformers.generation.utils import GreedySearchDecoderOnlyOutput
from transformers import GenerationConfig
import textwrap

DEVICE = 'cuda:0'

In [4]:
BASE_MODEL = r"model/llama2-7B-hf"

model = LlamaForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

tokenizer = LlamaTokenizer.from_pretrained(BASE_MODEL)

RuntimeError: CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [5]:
tokenizer.pad_token_id = (
    0  # unk. we want this to be different from the eos token
)
tokenizer.padding_side = "left"

NameError: name 'tokenizer' is not defined

In [ ]:
PROMPT_TEMPLATE_name = """
Your task is to generate a series of keywords of the venue based on its name, category, description, average price, customer reviews, etc. The generated keywords should be able to describe every aspect of the venue well.

### Example 1:

#### Input:
Venue Name: Chipotle Mexican Grill

#### Response:
meats, good service, chicken, well, lunch, cocktails, good for working, rice, lines, dinner, great value, burritos, onions, margaritas, dips, salsa, trendy, corn, good for a quick meal, barbacoa, black beans, red chili.

### Example 2:

#### Input:
Venue Name: Tony's Pizza

#### Response:
chicken, pizza, soup, cheese, city, great value, bacon, good for a quick meal, hot sauce, Sicilian, garlic knots, carmine.

### Example 3:

#### Input:
Venue Name: Panera Bread

#### Response:
restaurants, bar, alcohol, good service, breakfast food, sandwiches, soup, lunch, good for working, cake, bread, dinner, chili, trendy, good for a quick meal, cream cheese, iced green tea.

### Your Task:

#### Input:
{"[INPUT]"}

#### Response:
"""


PROMPT_TEMPLATE_name_cat_des_review = """
Your task is to generate a series of keywords of the venue based on its name, category, description, average price, customer reviews, etc. The generated keywords should be able to describe every aspect of the venue well.

### Example 1:

#### Input:
Venue Name: Chipotle Mexican Grill
Venue Category: Restaurant
Venue Description: Chipotle is a restaurant that prepares burritos and tacos at reasonable prices. Our Food With Integrity philosophy includes unprocessed, sustainable, nutritious, responsibly raised, and organic foods where possible.. 
Venue Reviews: 1. If you ask for double meat before the rice, sometimes they’ll forget to charge you extra., 2. Disrespectful staff. I asked for extra chicken and rice on my burrito. The rice made it in but the chicken was thrown in the bag with no lid. And they rolled their eyes when I asked! Never again!, 3. Good portions., 4. They're pretty speedy despite the insane lunch line!! 5. Great place for a quick chicken steak or pork bowl.

#### Response:
meats, good service, chicken, well, lunch, cocktails, good for working, rice, lines, dinner, great value, burritos, onions, margaritas, dips, salsa, trendy, corn, good for a quick meal, barbacoa, black beans, red chili.

### Example 2:

#### Input:
Venue Name: Tony's Pizza
Venue Category: Restaurant 
Venue Description: None
Venue Reviews: 1. What you order isn’t necessarily what you’ll get. Unless perhaps you order cramps and indigestion., 2. Love this spot :) cheap and good pizza, 3. Simpler than Carmine's but just as good if not better. , 4. Eat your upside down slice fuckin backwards , 5. If you get pizza on Graham Ave, you're either a Tony's guy or a Carmine's guy. I'm a Carmine's guy..

#### Response:
chicken, pizza, soup, cheese, city, great value, bacon, good for a quick meal, hot sauce, Sicilian, garlic knots, carmine.

### Example 3:

#### Input:
Venue Name: Panera Bread
Venue Category: Restaurant
Venue Description: From focusing on quality, clean ingredients to serving our food to you in a warm and welcoming environment, Panera Bread is committed to being an ally to our guests. That means crafting a menu of soups, salads and sandwiches that we are proud to feed our families.... 
Venue Reviews: 1. Easy off and on the highway, 2. I love the savory flavor of the French onion soup. My new fave is the cinnamon crunch bagel toasted with cream cheese, 3. Smoked chicken. Fresh avocado. Melted smoked Gouda. And freshly-baked Black Pepper Focaccia. Just a few good reasons to crave our Chipotle Chicken Avocado Melt. At Panera. Food as it should be. 

#### Response:
restaurants, bar, alcohol, good service, breakfast food, sandwiches, soup, lunch, good for working, cake, bread, dinner, chili, trendy, good for a quick meal, cream cheese, iced green tea.

### Your Task:

#### Input:
{"[INPUT]"}

#### Response:
"""

PROMPT_TEMPLATE_name_cat = """
Your task is to generate a series of keywords of the venue based on its name, category, description, average price, customer reviews, etc. The generated keywords should be able to describe every aspect of the venue well.

### Example 1:

#### Input:
Venue Name: Chipotle Mexican Grill
Venue Category: Restaurant

#### Response:
meats, good service, chicken, well, lunch, cocktails, good for working, rice, lines, dinner, great value, burritos, onions, margaritas, dips, salsa, trendy, corn, good for a quick meal, barbacoa, black beans, red chili.

### Example 2:

#### Input:
Venue Name: Tony's Pizza
Venue Category: Restaurant 

#### Response:
chicken, pizza, soup, cheese, city, great value, bacon, good for a quick meal, hot sauce, Sicilian, garlic knots, carmine.

### Example 3:

#### Input:
Venue Name: Panera Bread
Venue Category: Restaurant

#### Response:
restaurants, bar, alcohol, good service, breakfast food, sandwiches, soup, lunch, good for working, cake, bread, dinner, chili, trendy, good for a quick meal, cream cheese, iced green tea.

### Your Task:

#### Input:
{"[INPUT]"}

#### Response:
"""

PROMPT_TEMPLATE_name_cat_des = """
Your task is to generate a series of keywords of the venue based on its name, category, description, average price, customer reviews, etc. The generated keywords should be able to describe every aspect of the venue well.

### Example 1:

#### Input:
Venue Name: Chipotle Mexican Grill
Venue Category: Restaurant
Venue Description: Chipotle is a restaurant that prepares burritos and tacos at reasonable prices. Our Food With Integrity philosophy includes unprocessed, sustainable, nutritious, responsibly raised, and organic foods where possible.. 

#### Response:
meats, good service, chicken, well, lunch, cocktails, good for working, rice, lines, dinner, great value, burritos, onions, margaritas, dips, salsa, trendy, corn, good for a quick meal, barbacoa, black beans, red chili.

### Example 2:

#### Input:
Venue Name: Tony's Pizza
Venue Category: Restaurant 
Venue Description: None

#### Response:
chicken, pizza, soup, cheese, city, great value, bacon, good for a quick meal, hot sauce, Sicilian, garlic knots, carmine.

### Example 3:

#### Input:
Venue Name: Panera Bread
Venue Category: Restaurant
Venue Description: From focusing on quality, clean ingredients to serving our food to you in a warm and welcoming environment, Panera Bread is committed to being an ally to our guests. That means crafting a menu of soups, salads and sandwiches that we are proud to feed our families.... 

#### Response:
restaurants, bar, alcohol, good service, breakfast food, sandwiches, soup, lunch, good for working, cake, bread, dinner, chili, trendy, good for a quick meal, cream cheese, iced green tea.

### Your Task:

#### Input:
{"[INPUT]"}

#### Response:
"""

PROMPT_TEMPLATE_name_review = """
Your task is to generate a series of keywords of the venue based on its name, category, description, average price, customer reviews, etc. The generated keywords should be able to describe every aspect of the venue well.

### Example 1:

#### Input:
Venue Name: Chipotle Mexican Grill
Venue Reviews: 1. If you ask for double meat before the rice, sometimes they’ll forget to charge you extra., 2. Disrespectful staff. I asked for extra chicken and rice on my burrito. The rice made it in but the chicken was thrown in the bag with no lid. And they rolled their eyes when I asked! Never again!, 3. Good portions., 4. They're pretty speedy despite the insane lunch line!! 5. Great place for a quick chicken steak or pork bowl.

#### Response:
meats, good service, chicken, well, lunch, cocktails, good for working, rice, lines, dinner, great value, burritos, onions, margaritas, dips, salsa, trendy, corn, good for a quick meal, barbacoa, black beans, red chili.

### Example 2:

#### Input:
Venue Name: Tony's Pizza
Venue Reviews: 1. What you order isn’t necessarily what you’ll get. Unless perhaps you order cramps and indigestion., 2. Love this spot :) cheap and good pizza, 3. Simpler than Carmine's but just as good if not better. , 4. Eat your upside down slice fuckin backwards , 5. If you get pizza on Graham Ave, you're either a Tony's guy or a Carmine's guy. I'm a Carmine's guy..

#### Response:
chicken, pizza, soup, cheese, city, great value, bacon, good for a quick meal, hot sauce, Sicilian, garlic knots, carmine.

### Example 3:

#### Input:
Venue Name: Panera Bread
Venue Reviews: 1. Easy off and on the highway, 2. I love the savory flavor of the French onion soup. My new fave is the cinnamon crunch bagel toasted with cream cheese, 3. Smoked chicken. Fresh avocado. Melted smoked Gouda. And freshly-baked Black Pepper Focaccia. Just a few good reasons to crave our Chipotle Chicken Avocado Melt. At Panera. Food as it should be. 

#### Response:
restaurants, bar, alcohol, good service, breakfast food, sandwiches, soup, lunch, good for working, cake, bread, dinner, chili, trendy, good for a quick meal, cream cheese, iced green tea.

### Your Task:

#### Input:
{"[INPUT]"}

#### Response:
"""

PROMPT_TEMPLATE_name_des = """
Your task is to generate a series of keywords of the venue based on its name, category, description, average price, customer reviews, etc. The generated keywords should be able to describe every aspect of the venue well.

### Example 1:

#### Input:
Venue Name: Chipotle Mexican Grill
Venue Description: Chipotle is a restaurant that prepares burritos and tacos at reasonable prices. Our Food With Integrity philosophy includes unprocessed, sustainable, nutritious, responsibly raised, and organic foods where possible.. 

#### Response:
meats, good service, chicken, well, lunch, cocktails, good for working, rice, lines, dinner, great value, burritos, onions, margaritas, dips, salsa, trendy, corn, good for a quick meal, barbacoa, black beans, red chili.

### Example 2:

#### Input:
Venue Name: Tony's Pizza
Venue Description: None

#### Response:
chicken, pizza, soup, cheese, city, great value, bacon, good for a quick meal, hot sauce, Sicilian, garlic knots, carmine.

### Example 3:

#### Input:
Venue Name: Panera Bread
Venue Description: From focusing on quality, clean ingredients to serving our food to you in a warm and welcoming environment, Panera Bread is committed to being an ally to our guests. That means crafting a menu of soups, salads and sandwiches that we are proud to feed our families.... 

#### Response:
restaurants, bar, alcohol, good service, breakfast food, sandwiches, soup, lunch, good for working, cake, bread, dinner, chili, trendy, good for a quick meal, cream cheese, iced green tea.

### Your Task:

#### Input:
{"[INPUT]"}

#### Response:
"""

PROMPT_TEMPLATE_name_cat_review = """
Your task is to generate a series of keywords of the venue based on its name, category, description, average price, customer reviews, etc. The generated keywords should be able to describe every aspect of the venue well.

### Example 1:

#### Input:
Venue Name: Chipotle Mexican Grill
Venue Category: Restaurant
Venue Reviews: 1. If you ask for double meat before the rice, sometimes they’ll forget to charge you extra., 2. Disrespectful staff. I asked for extra chicken and rice on my burrito. The rice made it in but the chicken was thrown in the bag with no lid. And they rolled their eyes when I asked! Never again!, 3. Good portions., 4. They're pretty speedy despite the insane lunch line!! 5. Great place for a quick chicken steak or pork bowl.

#### Response:
meats, good service, chicken, well, lunch, cocktails, good for working, rice, lines, dinner, great value, burritos, onions, margaritas, dips, salsa, trendy, corn, good for a quick meal, barbacoa, black beans, red chili.

### Example 2:

#### Input:
Venue Name: Tony's Pizza
Venue Category: Restaurant 
Venue Reviews: 1. What you order isn’t necessarily what you’ll get. Unless perhaps you order cramps and indigestion., 2. Love this spot :) cheap and good pizza, 3. Simpler than Carmine's but just as good if not better. , 4. Eat your upside down slice fuckin backwards , 5. If you get pizza on Graham Ave, you're either a Tony's guy or a Carmine's guy. I'm a Carmine's guy..

#### Response:
chicken, pizza, soup, cheese, city, great value, bacon, good for a quick meal, hot sauce, Sicilian, garlic knots, carmine.

### Example 3:

#### Input:
Venue Name: Panera Bread
Venue Category: Restaurant
Venue Reviews: 1. Easy off and on the highway, 2. I love the savory flavor of the French onion soup. My new fave is the cinnamon crunch bagel toasted with cream cheese, 3. Smoked chicken. Fresh avocado. Melted smoked Gouda. And freshly-baked Black Pepper Focaccia. Just a few good reasons to crave our Chipotle Chicken Avocado Melt. At Panera. Food as it should be. 

#### Response:
restaurants, bar, alcohol, good service, breakfast food, sandwiches, soup, lunch, good for working, cake, bread, dinner, chili, trendy, good for a quick meal, cream cheese, iced green tea.

### Your Task:

#### Input:
{"[INPUT]"}

#### Response:
"""

In [5]:
templates = {'name': PROMPT_TEMPLATE_name,
             'name_review': PROMPT_TEMPLATE_name_review, 
             'name_des': PROMPT_TEMPLATE_name_des, 
             'name_cat': PROMPT_TEMPLATE_name_cat, 
             'name_cat_review': PROMPT_TEMPLATE_name_cat_review, 
             'name_cat_des': PROMPT_TEMPLATE_name_cat_des,
             'name_cat_des_review': PROMPT_TEMPLATE_name_cat_des_review}

In [6]:
from transformers import GenerationConfig

def generate_batch_responses(text_inputs, model, tokenizer, device, template, template_name, batch_size=8):
    try:
        with open('result/keyword_'+template_name+'.json', 'r') as jsonfile:
            responses = json.load(jsonfile)
            count = len(responses) - 1
    except:
        responses = []
        count = -1
        
    # Prepare batches of prompts
    for i in range(0, len(text_inputs), batch_size):
        if i <= count:
            continue
        batch_prompts = text_inputs[i:i + batch_size]
        prompts = [template.replace("[INPUT]", text).replace("'", '') for text in batch_prompts]
        
        # Tokenize all prompts in the current batch
        #encoding = tokenizer(prompts, padding=True, return_tensors="pt")
        encoding = tokenizer(prompts, padding=True, return_tensors="pt")
        input_ids = encoding["input_ids"].to(device)
        
        # Generate responses for the entire batch
        with torch.inference_mode():
            batch_responses = model.generate(
                input_ids=input_ids,
                max_new_tokens=128,  # Assuming 128 is the desired maximum length
                temperature=0.1,
                top_p=0.75,
                repetition_penalty=1.1,
                #pad_token_id=tokenizer.eos_token_id,  # Ensure padding with the EOS token
                return_dict_in_generate=True,
                output_scores=True,
            )
        
        # Decode each response in the batch
        for response in batch_responses.sequences:
            decoded_output = tokenizer.decode(response)
            split_response = decoded_output.split("#### Response:")[4].strip()
            wrapped_response = "\n".join(textwrap.wrap(split_response))
            responses.append(wrapped_response)
            
        if i % 10 == 0:
            filename = "result/keyword_" + template_name + ".json"
            with open(filename, "w") as json_file:
                json.dump(responses, json_file)

    return responses

for template_name, template in templates.items():
    
    # if template_name not in ['name_des_review_keyword']:
    #     continue
    
    # read the target api dataset
    with open('data/keywords_'+template_name+'.json', 'r') as jsonfile:
        prompt_venue_text = json.load(jsonfile)
    
    text_inputs = []
    labels = []

    for idx, venue in prompt_venue_text.items():
        text_inputs.append(venue['prompt'])
        labels.append(venue['label'])
    
    generated_formatted_venue_text = generate_batch_responses(
        text_inputs=text_inputs,
        model=model,
        tokenizer=tokenizer,
        device=DEVICE,
        template = template,
        template_name = template_name,
        batch_size=8  # Adjust batch size as needed based on available memory
    )

    # Save the generated responses to a file
    filename = "result/keyword_" + template_name + ".json"
    with open(filename, "w") as json_file:
        json.dump(generated_formatted_venue_text, json_file)

/home/bangzhao/.local/lib/python3.9/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
